Purpose: pipeline so that given a list of CCSs, for each label where the 5p barcode, 3p barcode, and adapter lie 

In [274]:
import CCS_Filter as cf
import numpy as np
import random as rd

In [275]:
#adaptation of CCS_Filter's score alignment, except this time 
#RETURNS list of score, traceback alignment (just reference portion)
#FOR NOW: linear gap penalty d
#parameters: gap penalty d, barcode/reference(vertical), seq y(horizontal),
    #score matrix, reference character
#reference character: when returning alignment, replace all characters in seq y
    # with the reference character. Then only return the line of the alignment for 
    # the reference
    #example of return: 55555--5----------------
    #if the reference is the 5p barcode
#initialization: 0s in first row; (i,0) = -i*d
#Traceback favors iy>m>ix becuase more likely to have gaps in ref than 
    #y since reference shorter than y
    

def glocal_alignment(gap_penalty, sequence, reference, score_dict, reference_char):
    gap_penalty = np.abs(gap_penalty)
    #matrix
    num_rows = len(reference) +1
    num_cols = len(sequence) +1
    F = np.zeros((num_rows, num_cols))
    T = np.zeros((num_rows,num_cols)) #traceback
    #labels for tracing, prioritizing iy>m>ix
    IY, M, IX = 3, 2, 1
    T_list = [IY, M, IX]
    #initialization",
    #row 0 already initialized to zeros
    #keep T[0][j] as zeros because can stop anywhere there
    for i in np.arange(1,num_rows):
        F[i][0] = -1*i*gap_penalty
        T[i][0] = IX
        

    for i in np.arange(1,num_rows): # i represents the row
        for j in np.arange(1,num_cols): #j represents the col
#            print('i is', i)
#            print('j is', j)
#            print("looking at ref(x) char", reference[i-1])
#            print("looking at seq(y) char", sequence[j-1])
            m = F[i-1][j-1] + score_dict[(reference[i-1],sequence[j-1])]
            ix = F[i-1][j] - gap_penalty
            iy = F[i][j-1] - gap_penalty
#            print('m is', m)
#            print('ix is',ix)
#            print('iy is', iy)
            F[i][j], T[i][j] = max(list(zip([iy,m,ix],T_list)))
#            print("T entry is: ",T[i][j])
    #now matrix is filled"
    last_row_array = np.array(F[num_rows-1])
    max_indices = np.where(last_row_array == max(F[num_rows-1]))
    max_indices_list = []
    for elem in max_indices:
        for entry in elem:
            max_indices_list.append(entry)
#    print('max_indices is:',max_indices_list)


    score_list = [F[num_rows-1][i] for i in max_indices_list]
#    print("T is: ")
#    print(T)
    x_alignments = []
    y_alignments = []
    for trace_start_col in max_indices_list:
        #start traceback
        x_align = [] #reference
        y_align = [] #seq
        seq_length = len(sequence)
        y = sequence
        x = [reference_char]*len(reference)
        i_fin, j_fin = num_rows-1, trace_start_col
        i = i_fin
        j = j_fin
        traceback = T[i][j]
        current = ''
        if j_fin < seq_length:
            for n in np.arange(seq_length - j_fin):
                x_align.append('-')
                y_align.append(y[seq_length-n-1])
        if traceback == M:
            i -= 1
            j -= 1
            y_align.append(y[j])
            x_align.append(x[i])
        elif current == IY:
            j -= 1
            x_align.append('-')
            y_align.append(y[j])
        else: #current == IX
            i -= 1
            x_align.append(x[i])
            y_align.append('-')


        current = T[i][j]

        while i>0 and j>0:
            if current == M:
                i -= 1
                j -= 1
                y_align.append(y[j])
                x_align.append(x[i])
            elif current == IY:
                j -= 1
                x_align.append('-')
                y_align.append(y[j])
            else: #current == IX
                i -= 1
                x_align.append(x[i])
                y_align.append('-')
            current = T[i][j]

        if j >0:
            while j >0:
                j -= 1
                x_align.append('-')
                y_align.append(y[j])


        x_align.reverse()
        y_align.reverse()
        x_alignments.append(x_align)
        y_alignments.append(y_align)
        
    return_list = list(zip(score_list,x_alignments))
    
#    x_align_str = "".join(x_align)
 #   y_align_str = "".join(y_align)
    
    
#    print('y(seq): ', y_align)
 #   print('x(ref): ', x_align)
 #   print('Returning: ',return_list)
    return return_list
reference = "ACGT"
ref_rc = cf.gen_rev_complement(reference)
seq_0 = "ACGATAGA"
seq0_rc = cf.gen_rev_complement(seq_0)
seq_1 = "AAGACAGT"
seq1_rc = cf.gen_rev_complement(seq_1)
seq_2 = "AGGACGCTAA"
seq2_rc = cf.gen_rev_complement(seq_2)
glocal_alignment(gap_penalty=1, sequence=test_3C01_strings[0], reference=three_adapter, score_dict=cf.score_dict, reference_char = 'R')
a=1

In [303]:
#test strings
nucleotides = cf.nucleotides

fivePBarcode = cf.barcodes_list[0]
fivePBarcodeRC = cf.gen_rev_complement(fivePBarcode)
barcode2 = cf.barcode_2
barcode3 = cf.barcode_3
barcode4 = cf.barcode_4
barcode2RC = cf.barcode_2_rc
barcode3RC = cf.gen_rev_complement(barcode3)
barcode4RC = cf.gen_rev_complement(barcode4)
#adapter
two_adapter = ''
three_adapter = ''
four_adapter = ''
#/tier2/deweylab/data/thomson_lab/Pacbio/2_B01/m54178_180915_120213.adapters.fasta
with open('two_B01.adapters.fasta') as f: 
    for line in f: 
        line = line.strip()
        if line[0]!='>':
            two_adapter = line

#/tier2/deweylab/data/thomson_lab/Pacbio/3_C01/m54178_180916_082219.adapters.fasta
with open('three_C01.adapters.fasta') as f: 
    for line in f: 
        line = line.strip()
        if line[0]!='>':
            three_adapter = line

#/tier2/deweylab/data/thomson_lab/Pacbio/4_D01/m54178_180917_044252.adapters.fasta            
with open('four_D01.adapters.fasta') as f: 
    for line in f: 
        line = line.strip()
        if line[0]!='>':
            four_adapter = line
two_adapter_RC = cf.gen_rev_complement(two_adapter)
three_adapter_RC = cf.gen_rev_complement(three_adapter)
four_adapter_RC = cf.gen_rev_complement(four_adapter)
print("len two_adapter:", len(two_adapter))
print("len three_adapter:", len(three_adapter))
print("len four_adapter:", len(four_adapter))
print("len 5p barcode:", len(fivePBarcode))
print("len barcode2:", len(barcode2))
print("len barcode3:", len(barcode3))
print("len barcode4:", len(barcode4))

print("len two_adapter + 3p barcode:", len(two_adapter) + len(barcode2))
print("len three_adapter + 5p barcode:", len(three_adapter)+len(fivePBarcode))
print("len four_adapter + 5p barcode:", len(four_adapter)+len(fivePBarcode))
print("total: ", len(two_adapter) + len(barcode2) + len(fivePBarcode))

len two_adapter: 45
len three_adapter: 45
len four_adapter: 45
len 5p barcode: 25
len barcode2: 71
len barcode3: 71
len barcode4: 71
len two_adapter + 3p barcode: 116
len three_adapter + 5p barcode: 70
len four_adapter + 5p barcode: 70
total:  141


In [287]:
#makes substituted sequence string given seq and rand nuc list
def generate_sub_string(seq, rand_nuc_list, rand_subs_list):
    seq_list = list(seq)
#    print("length seq:", len(seq_list))
#    print("length rand nuc list", len(rand_nuc_list))
#    print("length rand subs list", len(rand_subs_list))
    for i in np.arange(len(rand_nuc_list)):
#        print("rand_subs_list[" + str(i) + "]:" + str(rand_subs_list[i]))
#        print("seq_list[rand_subs_list[i]]:", seq_list[rand_subs_list[i]])
#        print("rand_nuc_list[i]", rand_nuc_list[i])
        seq_list[rand_subs_list[i]] = rand_nuc_list[i]
    sub_str = "".join(seq_list)
#    print("success!! returning. ")
#    print()
    return sub_str

#makes inserted sequence given seq and rand nuc list
def generate_insert_string(seq, rand_nuc_list, rand_subs_list):
    seq_list = list(seq)
    for i in np.arange(len(rand_nuc_list)):
        index = rand_subs_list[i] + i
        seq_list.insert(index, rand_nuc_list[i])
    insert_str = "".join(seq_list)
    return insert_str

#makes deleted sequence given seq and rand nuc list
def generate_delete_string(seq, rand_subs_list):
    seq_list = list(seq)
#    print("length seq:", len(seq_list))
#    print("length rand subs list", len(rand_subs_list))
    for i in np.arange(int(len(rand_subs_list))):    
#        print("rand_subs_list[" + str(i) + "]:" + str(rand_subs_list[i]))
        index = rand_subs_list[i] -i
#        print("seq_list[rand_subs_list[i]]:", seq_list[index])
        del seq_list[index]
    del_str = "".join(seq_list)
    return del_str
    

In [288]:
def gen_test_strings(barcode, barcodeRC, adapter, adapterRC):    
    test_strings = []
    #First: exact matches
    '''#Ar3r5r
    test_ar3r5r = adapterRC + barcodeRC + fivePBarcodeRC
    test_strings.append(test_ar3r5r)
    #A53A
    test_a53a = adapter +"".join(rd.choices(nucleotides, k=50))+ fivePBarcode+ "".join(rd.choices(nucleotides, k=100)) + barcode+ "".join(rd.choices(nucleotides, k=10)) + adapter
    test_strings.append(test_a53a)
    #5rA3Ar
    test_5ra3ar = fivePBarcodeRC + adapter + "".join(rd.choices(nucleotides, k=10)) + barcode+"".join(rd.choices(nucleotides, k=100)) +adapterRC
    test_strings.append(test_5ra3ar)
    #start5
    test_start5= fivePBarcode + "".join(rd.choices(nucleotides, k=(1000)))
    test_strings.append(test_start5)
    #A_rand_A
    test_a_rand_a = adapter + "".join(rd.choices(nucleotides, k=900)) + adapter
    test_strings.append(test_a_rand_a)
    #rand
    test_random="".join(rd.choices(nucleotides, k=(1000)))
    test_strings.append(test_random)
    #end in 3
    test_endIn3pBarcode= "".join(rd.choices(nucleotides, k=1000)) + barcode
    test_strings.append(test_endIn3pBarcode)
    '''
    #########
    #Next: mutated matches
    bar2_list = list(barcode)
    rand_subs_list = rd.choices(np.arange(len(barcode)), k=int(len(barcode)/10)) #pacbio MUCH more accurate
    #here, error in sequencing is 10%
    #pacbio has error of about 0.001%
    #source: https://www.pacb.com/uncategorized/a-closer-look-at-accuracy-in-pacbio/
    
    rand_nuc_list = rd.choices(nucleotides, k=len(rand_subs_list))
    #3p barcode
    rand_subs_list = rd.choices(np.arange(len(barcode)), k=int(len(barcode)/10))
    rand_nuc_list = rd.choices(nucleotides, k=len(rand_subs_list))
    sub_bar = generate_sub_string(barcode, rand_nuc_list, rand_subs_list)
    sub_bar_rc = cf.gen_rev_complement(sub_bar)
    insert_bar = generate_insert_string(barcode,rand_nuc_list, rand_subs_list)
    insert_bar_rc = cf.gen_rev_complement(insert_bar)
    del_bar = generate_delete_string(barcode, rand_subs_list)
    del_bar_rc = cf.gen_rev_complement(del_bar)
    
    #adapter
    rand_subs_list = rd.choices(np.arange(len(adapter)), k=int(len(adapter)/10))
    rand_nuc_list = rd.choices(nucleotides, k=len(rand_subs_list))
    sub_adapter = generate_sub_string(adapter, rand_nuc_list, rand_subs_list)
    sub_adapter_rc = cf.gen_rev_complement(sub_adapter)
    insert_adapter = generate_insert_string(adapter, rand_nuc_list, rand_subs_list)
    insert_adapter_rc = cf.gen_rev_complement(insert_adapter)
    del_adapter = generate_delete_string(adapter, rand_subs_list)
    del_adapter_rc = cf.gen_rev_complement(del_adapter)
    
    #5p barcode
    rand_subs_list = rd.choices(np.arange(len(fivePBarcode)), k=int(len(fivePBarcode)/10))
    rand_nuc_list = rd.choices(nucleotides, k=len(rand_subs_list))
    sub_5 = generate_sub_string(fivePBarcode, rand_nuc_list, rand_subs_list)
    sub_5_rc = cf.gen_rev_complement(sub_5)
    insert_5 = generate_insert_string(fivePBarcode, rand_nuc_list, rand_subs_list)
    insert_5_rc = cf.gen_rev_complement(insert_5)
    del_5 = generate_delete_string(fivePBarcode, rand_subs_list)
    del_5_rc = cf.gen_rev_complement(del_5)
    
    
    #####
    #test strings
    #####
    #Ar3r5r
    test_ar3r5r_sub = sub_adapter_rc + sub_bar_rc + sub_5_rc
    test_ar3r5r_insert = insert_adapter_rc + insert_bar_rc + insert_5_rc
    test_ar3r5r_del = del_adapter_rc + del_bar_rc + del_5_rc
    test_strings.append(test_ar3r5r_sub)
    test_strings.append(test_ar3r5r_insert)
    test_strings.append(test_ar3r5r_del)
    #A53A
    test_a53a_sub = sub_adapter +"".join(rd.choices(nucleotides, k=50))+ sub_5 + "".join(rd.choices(nucleotides, k=100)) + sub_bar+ "".join(rd.choices(nucleotides, k=10)) + sub_adapter
    test_a53a_insert = insert_adapter +"".join(rd.choices(nucleotides, k=50))+ insert_5 + "".join(rd.choices(nucleotides, k=100)) + insert_bar+ "".join(rd.choices(nucleotides, k=10)) + insert_adapter
    test_a53a_del = del_adapter +"".join(rd.choices(nucleotides, k=50))+ del_5 + "".join(rd.choices(nucleotides, k=100)) + del_bar+ "".join(rd.choices(nucleotides, k=10)) + del_adapter
    test_strings.append(test_a53a_sub)
    test_strings.append(test_a53a_insert)
    test_strings.append(test_a53a_del)

    #5rA3Ar
    test_5ra3ar_sub = sub_5_rc + sub_adapter + "".join(rd.choices(nucleotides, k=10)) + sub_bar +"".join(rd.choices(nucleotides, k=100)) +sub_adapter_rc
    test_5ra3ar_insert = insert_5_rc + insert_adapter + "".join(rd.choices(nucleotides, k=10)) + insert_bar +"".join(rd.choices(nucleotides, k=100)) +insert_adapter_rc
    test_5ra3ar_del = del_5_rc + del_adapter + "".join(rd.choices(nucleotides, k=10)) + del_bar +"".join(rd.choices(nucleotides, k=100)) +del_adapter_rc
    test_strings.append(test_5ra3ar_sub)
    test_strings.append(test_5ra3ar_insert)
    test_strings.append(test_5ra3ar_del)
    return test_strings

#actual test strings
test_2B01_strings = gen_test_strings(barcode2, barcode2RC, two_adapter, two_adapter_RC)
test_3C01_strings = gen_test_strings(barcode3, barcode3RC, three_adapter, three_adapter_RC)
test_4D01_strings = gen_test_strings(barcode4, barcode4RC, four_adapter, four_adapter_RC)
#exact matches: Ar3r5r, A53A, 5rA3Ar, ,start5, A_rand_A, rand, endIn3

In [295]:
#given a sequence, and a list of lists of [reference, reference character], 
#annotates the sequence with the location(s) of the reference(s)
#does references and their reverse complements, with the RCs having "r" 
    #appended to the reference character
#only includes reference(s) with a score significance of >0.99999999
#RETURN: of the references that pass the score significance filter, 
    #returns fusion of the alignment strings returned by glocal alignment function

#parameters
    #sequence: string
    #ref_list: list of lists
        #each entry is [reference(string), reference character]
def annotate_seq(sequence, ref_list):
    fused_list = ['-']*len(sequence) 
    for refPair in ref_list:
        print("ref is" ,refPair[1])
        ref = refPair[0]
        ref_char = refPair[1]
        refRC = cf.gen_rev_complement(ref)
        refRC_char = ref_char + 'r'
        ref_list = glocal_alignment(gap_penalty=1, sequence=sequence,
                                              reference=ref, score_dict = cf.score_dict,
                                     reference_char = ref_char)
#        print("ref_list:",ref_list)
        ref_scores = []
        ref_alignments = []
        for pair in ref_list:
            ref_scores.append(pair[0])
            ref_alignments.append(pair[1])
#        print("ref_scores",ref_scores)
            
        rc_list = glocal_alignment(gap_penalty=1, sequence=sequence,
                                              reference=refRC, score_dict = cf.score_dict,
                                              reference_char = refRC_char)
 #       print("rc_list",rc_list)
        rc_scores = []
        rc_alignments = []
        for pair in rc_list:
            rc_scores.append(pair[0])
            rc_alignments.append(pair[1])
#        print("rc_scores",rc_scores)
            
        ref_scoreSig_list = [cf.calc_score_sig(ref_score) for ref_score in ref_scores]
        rc_scoreSig_list = [cf.calc_score_sig(rc_score) for rc_score in rc_scores]
        print("ref score sigs:", ref_scoreSig_list)
        print("ref RC score sigs:",rc_scoreSig_list)
#        print()
#        print()
        THRESHOLD = 0.999999999 #(9) 9's
        for i in np.arange(len(ref_scoreSig_list)):
            if ref_scoreSig_list[i]>THRESHOLD:
                fuse(fused_list,ref_alignments[i])
        for i in np.arange(len(rc_scoreSig_list)):
            if rc_scoreSig_list[i]>THRESHOLD:
                fuse(fused_list,rc_alignments[i])
        
#    print("fused_list: ",fused_list)
    return_str = "".join(fused_list)
#    print("string version of fused_list\n", return_str)
    return return_str
    
#print(cf.calc_score_sig(45))
#print(cf.calc_score_sig(-3))

In [296]:
#given current fused list and list of what to fuse, fuses the "to_fuse" contents to the fused list
def fuse(fused, to_fuse):
    len_fused = len(fused)
    for i in np.arange(len_fused):
        if to_fuse[i] != '-':
            if fused[i] != '-':
                fused[i] = '!'
            else:
                fused[i] = to_fuse[i]

In [300]:
#test it!

#test fuse:
def test_fuse(filler):
    filler = '-'
    fused = [filler] * 5
    toFuse0 = [filler,filler,filler,'0','0']
    fuse(fused,toFuse0)
    toFuse1 = [filler,'1','1','1',filler]
    fuse(fused, toFuse1)
    assert fused == [filler, '1', '1', '1', '0']
#test_fuse('-')

#test annotate_seq
ref_list_2 = [[barcode2,'2'], [two_adapter,'A'],[fivePBarcode,'5'] ]
ref_list_3 = [[barcode3,'3'], [three_adapter,'A'], [fivePBarcode,'5']]
ref_list_4 = [[barcode4,'4'], [four_adapter,'A'], [fivePBarcode,'5']]
def test_annotateseq(ref_list, seq_list):
    for seq in seq_list:
        annotate_seq(seq, ref_list)
#        print(annotate_seq(seq, ref_list))
        print()
test_annotateseq(ref_list_2, test_2B01_strings)
#  0       1     2        3         4       5      6
#Ar3r5r, A53A, 5rA3Ar, ,start5, A_rand_A, rand, endIn3

ref is 2
ref score sigs: [3.033024161965527e-09]
ref RC score sigs: [1.0, 1.0]
ref is A
ref score sigs: [0.9157761915991026, 0.9157761915991026]
ref RC score sigs: [1.0]
ref is 5
ref score sigs: [0.9954419611263292, 0.9954419611263292]
ref RC score sigs: [0.9999999999743453, 0.9999999999743453]

ref is 2
ref score sigs: [1.115787235902589e-09, 1.115787235902589e-09]
ref RC score sigs: [1.0]
ref is A
ref score sigs: [0.9157761915991026]
ref RC score sigs: [1.0]
ref is 5
ref score sigs: [0.9954419611263292]
ref RC score sigs: [0.9999999999743453, 0.9999999999743453]

ref is 2
ref score sigs: [1.115787235902589e-09]
ref RC score sigs: [1.0]
ref is A
ref score sigs: [0.16607476979884608, 0.16607476979884608]
ref RC score sigs: [1.0]
ref is 5
ref score sigs: [0.9954419611263292]
ref RC score sigs: [0.999999999996528]

ref is 2
ref score sigs: [1.0]
ref RC score sigs: [0.0004933956569012966]
ref is A
ref score sigs: [1.0, 1.0]
ref RC score sigs: [0.9157761915991026, 0.9157761915991026, 0.915